In [2]:
#import libraries
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import wordcloud 
import seaborn as sns
import nltk
import emoji
nltk.download('punkt')
nltk.download('punkt_tab')
import io
import unicodedata
import re
import string
from numpy import linalg
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import webtext
from nltk.stem.porter import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('vader_lexicon')
import requests
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

#The details of post and the API endpoint
ACCESS_TOKEN = 'EABCH0JqmrmABO8T8fMAq2ZBWZAfRCbv7Umko6GmV2J8X2sLzuaRfSS31RU16ZBnzAxdEIpioYdFhTm4kgvpjpPU3GrZBfN5DFImvFhEV9P68Xg4oM8w1APUfGJQndrZB7c4hPFvGGxXHDW657Ezq5qZAd840VpAZBBoaqmh6ADM5tDuBG6Hz6NZBEEnDC7OBm58ZAG7l1fUeFfyg1UJf9I74YlJiMHqZCEZCha7KUgF8uAOU4xSjx4Cs25KZAsMRV0ZBB5AZDZD'
POST_ID = '18KCp9sP6E'
url = 'https://graph.facebook.com/v22.0/18KCp9sP6E/comments?access_token=EABCH0JqmrmABO8T8fMAq2ZBWZAfRCbv7Umko6GmV2J8X2sLzuaRfSS31RU16ZBnzAxdEIpioYdFhTm4kgvpjpPU3GrZBfN5DFImvFhEV9P68Xg4oM8w1APUfGJQndrZB7c4hPFvGGxXHDW657Ezq5qZAd840VpAZBBoaqmh6ADM5tDuBG6Hz6NZBEEnDC7OBm58ZAG7l1fUeFfyg1UJf9I74YlJiMHqZCEZCha7KUgF8uAOU4xSjx4Cs25KZAsMRV0ZBB5AZDZD'

#getting the comments as row http responses and stored in a response object
response = requests.get(url)

#Error handling in a user friendly way especially when fetching comments
if response.status_code == 200:
    print('Server error, Retrying...')
if response.status_code == 400:
    print('Issue with the request like invalid or expires access token')
if response.status_code == 401:
    print('Invalid access token. Please generate a new one')
if response.status_code == 403:
    print('Permission denied. Check your app permissions')
if response.status_code == 404:
    print('Post not found. Please verify the post ID')
else:
    print('Error fetching comments', response.json())

#converting from raw object to a jason object
comments_data = response.json()
comments = []
#iterate through json object and add comments to a comments list
for comment in comments_data.get('data',[]):
    comments.append(comment['message'])

#Since fb provide lismited API privilages, not able to fetch the all the comments at once
while 'next' in comments_data.get('paging', {}):
    next_page = comments_data['paging']['next']
    response = requests.get(next_page)
    comments_data =response.json()
    for comment in comments_data.get('data',[]):
        comments.append(comment['message'])

#Analyze Sarcasm using deep learning
#Load a pre-trained sarcasm detection BERT based model
model_name = "mrm8488/bert-mini-finetuned-sarcasm-detection"
tokenizer = AutoTokenizer.from_pretrained(model_name) 

#Ensure model runs on GPU if available
device = torch.device('cuda' if torch.cuda_is_available() else cpu) 
model.to(device) 

#now, comments list has all the comments.
for comment in comments:
    #Define a function to detect sarcasm
    def detect_sarcasm(text):
        inputs = tokenizer(text, return_tensors = 'pt', truncation = True, padding = True)
        inputs = {key:val.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim =1).item()

    return 'Sarcastic' if prediction = 1 else 'Not Sarcastic'
        
    
    #extracct emojis and convert it to a text description and combine all the elements of the sentence
    comment_with_emojis =emoji.demojize(comment, delimeter =('','')) 
    
    #extract hashtags with camel case 
    hashtags_word_list = re.findall(r'#([A-Z]?[a-z]+|[A-Z]+(?![a-z]))+',comment_with_emojis)
    full_sentence = comment_with_emojis + ''.join(hashtags_word_list)
    sentiment_score=sid.polarity_score(full_sentence)
    
    #calculate sentinmenst for each category and updating te dictionary
    sentiment_results = {'Positive':0, 'Negative':0, 'Neutral':0}
    if sentiment_score['compound'] > 0.05:
        sentiment_results['Positive'] += 1
    if sentiment_score['compound']< -0.05:
        sentiment_results['Negative'] += 1
    else:
        sentiment_results['Neutral'] += 1
    print(comment_with_emojis)
    print(sorted(sentiment_score))
    print('/n')

#Taking percentages 
total_positive_comments = sentiment_results['Positive'] 
total_negative_comments = sentiment_results['Negative']
total_neutral_comments = sentiment_results['Neutral']
total_comments = total_positive_comments + total_negative_comments + total_neutral_commments
positive_percentage =(total_positive_comments/total_comments)*100
negative_percentage = (total_negative_comments/total_comments)*100
neutral_percentage = (total_neutral_comments/total_comments)*100

print('Percentage of positive commnets: ', positive_percentage,'%')
print('Percentage of negative comments: ', negative_percentage, '%')
print('Percentage of neutral comments: ', neutral_percentage, '%')

#data visualization(bar graph)
sentiments = ['Positive','Negative','Neutral']
Counts = [total_positive_comments, total_negative_comments,total_neutral_comments]
sns.barplot(x=sentiments, y=counts, palette =['green', 'gray', 'red'])

#labels
plt.xlabel('Sentiments')
plt.ylabel('Counts')
plt.title('Sentiment Analysis')
plt.show()

#Analyze Sarcasm using deep learning
#Load a pre-trained sarcasm detection BERT based model
model_name = "mrm8488/bert-mini-finetuned-sarcasm-detection"
tokenizer = AutoTokenizer.from_pretrained(model_name) 

#Ensure model runs on GPU if available
device = torch.device('cuda' if torch.cuda_is_available() else cpu) 
model.to(device) 

#Define a function to detect sarcasm
def detect_sarcasm(text):
    inputs = tokenizer(text, return_tensors = 'pt', truncation = True, padding = True)
    inputs = {key:val.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim =1).item()

    return 'Sarcastic' if prediction = 1 else 'Not Sarcastic'
        




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ravee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ravee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ravee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ravee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ravee\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


ModuleNotFoundError: No module named 'transformers'